# Smooth manifolds, charts and scalar fields

This woksheet accompanies the lecture
[Symbolic tensor calculus on manifolds](http://sagemanifolds.obspm.fr/jncf2018/) at JNCF 2018.

Click [here](https://raw.githubusercontent.com/sagemanifolds/SageManifolds/master/Worksheets/JNCF2018/jncf18_scalar.ipynb) to download the worksheet file (ipynb format). To run it, you must start SageMath with the Jupyter notebook, via the command `sage -n jupyter`

*NB:* a version of SageMath at least equal to 7.5 (8.2 for the SymPy part) is required to run this worksheet:

In [1]:
version()

'SageMath version 8.2.beta3, Release Date: 2018-01-17'

First we set up the notebook to use LaTeX display:

In [2]:
%display latex

## Starting with a manifold

Manifolds are constructed via the global function `Manifold`:

In [3]:
M = Manifold(2, 'M')
print(M)

2-dimensional differentiable manifold M


By default, `Manifold` returns a manifold over $\mathbb{R}$:

In [4]:
M.base_field()

Real Field with 53 bits of precision

Other base fields must be specified with the optional keyword `field`, like
```python
M = Manifold(2, 'M', field='complex')
```
We may check that $M$ is a topological space:

In [5]:
M in Sets().Topological()

True

Actually, $M$ belongs to the following categories:

In [6]:
M.categories()

[Category of smooth manifolds over Real Field with 53 bits of precision,
 Category of differentiable manifolds over Real Field with 53 bits of precision,
 Category of manifolds over Real Field with 53 bits of precision,
 Category of topological spaces,
 Category of sets,
 Category of sets with partial maps,
 Category of objects]

As we can see, by default, `Manifold` constructs a smooth manifold.
If one would like to stick to the topological level, one should write
```python
M = Manifold(2, 'M', structure='topological')
```
Smooth manifolds are implemented by the Python class `DifferentiableManifold`:

In [7]:
type(M)

<class 'sage.manifolds.differentiable.manifold.DifferentiableManifold_with_category'>

The type of `M` appears as `DifferentiableManifold-with-category` because it is actually a subclass of `DifferentiableManifold`, which is dynamically generated by SageMath's [category mechanism](http://doc.sagemath.org/html/en/reference/categories/sage/categories/primer.html):

In [8]:
isinstance(M, 
       sage.manifolds.differentiable.manifold.DifferentiableManifold)

True

The class `DifferentiableManifold` inherits from `TopologicalManifold`:

In [9]:
isinstance(M, sage.manifolds.manifold.TopologicalManifold)

True

## Coordinate charts

We declare a chart, along with the symbols used to denote the coordinates (here $x=x^0$ and $y=x^1$) by

In [10]:
U = M.open_subset('U')
XU.<x,y> = U.chart()
XU

Chart (U, (x, y))

Open subsets are implemented by a (dynamically generated) subclass of
`DifferentiableManifold`, since they are manifolds in their own:

In [11]:
isinstance(U, 
       sage.manifolds.differentiable.manifold.DifferentiableManifold)

True

Points on $M$ are created by passing their coordinates in a given chart:

In [12]:
p = U((1,2), chart=XU, name='p')
print(p)

Point p on the 2-dimensional differentiable manifold M


The syntax `U(...)` used to create $p$ as an element of $U$ reflects the 
**parent/element pattern** employed in SageMath; indeed $U$ is the parent of $p$:

In [13]:
p.parent()

Open subset U of the 2-dimensional differentiable manifold M

Points are implemented by the class `ManifoldPoint`.
The principal attribute of this class is a Python dictionary storing the point's coordinates
in various charts:

In [14]:
p._coordinates

{Chart (U, (x, y)): (1, 2)}

Of course, we can recover the point's coordinates by letting the chart act on the point:

In [15]:
XU(p)

(1, 2)

Let us  introduce a second chart on $M$:

In [16]:
V = M.open_subset('V')
XV.<xp,yp> = V.chart("xp:x' yp:y'")
XV

Chart (V, (xp, yp))

and declare that $M$ is covered by only two charts, i.e. that $M=U\cup V$:

In [17]:
M.declare_union(U,V)

In [18]:
M.atlas()

[Chart (U, (x, y)), Chart (V, (xp, yp))]

### Transition map

We define the transition map `XU` $\to$ `XV` on $W = U\cap V$ as follows:

In [19]:
XU_to_XV = XU.transition_map(XV, 
                             (x/(x^2+y^2), y/(x^2+y^2)), 
                             intersection_name='W',
                             restrictions1= x^2+y^2!=0, 
                             restrictions2= xp^2+yp^2!=0)
XU_to_XV.display()

xp = x/(x^2 + y^2)
yp = y/(x^2 + y^2)

The value of the argument `restrictions1` means that
$W = U\setminus \{S\}$, where $S$ is the point of coordinates $(x,y)=(0,0)$,
while the value of `restrictions2` means that
$W = V\setminus \{N\}$, where $N$ is the point of coordinates $(x',y')=(0,0)$.
Since $M=U\cup V$, we have then
$$
    U = M \setminus \{N\},\qquad
    V = M \setminus \{S\},\quad\mbox{and}\quad
    W = M \setminus \{N, S\} .
$$

The transition map `XV` $\to$ `XU`  is obtained by computing the inverse of the one defined above:

In [20]:
XU_to_XV.inverse().display()

x = xp/(xp^2 + yp^2)
y = yp/(xp^2 + yp^2)

At this stage, the smooth manifold $M$ is fully specified, being covered by
one atlas with all transition maps specified. One may have recognized that
$M$ is nothing but the **2-dimensional sphere**:
$$
    M = \mathbb{S}^2 ,
$$
with `XU` (resp. `XV`) being
the chart of \defin{stereographic coordinates}\index{stereographic!coordinates}
from the North pole $N$ (resp. the South pole $S$).

Since the transition maps have been defined,
we can ask for the coordinates $(x',y')$ of the point $p$:

In [21]:
XV(p)

(1/5, 2/5)

This operation has updated the dictionary `_coordinates`:

In [22]:
p._coordinates

{Chart (V, (xp, yp)): (1/5, 2/5), Chart (U, (x, y)): (1, 2)}

## Smooth maps

As a example of smooth map, let us consider the canonical embedding of $M=\mathbb{S}^2$ 
into $\mathbb{R}^3$. We need first to introduce $\mathbb{R}^3$ as a 3-dimensional smooth
manifold, endowed with a single chart:

In [23]:
R3 = Manifold(3, 'R^3', r'\mathbb{R}^3')
XR3.<X,Y,Z> = R3.chart()
XR3

Chart (R^3, (X, Y, Z))

The embedding $\Phi: \mathbb{S}^2 \to \mathbb{R}^3$ is then defined in terms of its coordinate expression in the two charts covering $M=\mathbb{S}^2$:

In [24]:
Phi = M.diff_map(R3, {(XU, XR3): 
                       [2*x/(1+x^2+y^2), 2*y/(1+x^2+y^2),
                        (x^2+y^2-1)/(1+x^2+y^2)],
                      (XV, XR3): 
                       [2*xp/(1+xp^2+yp^2), 2*yp/(1+xp^2+yp^2),
                        (1-xp^2-yp^2)/(1+xp^2+yp^2)]},
                  name='Phi', latex_name=r'\Phi')
Phi.display()

Phi: M --> R^3
on U: (x, y) |--> (X, Y, Z) = (2*x/(x^2 + y^2 + 1), 2*y/(x^2 + y^2 + 1), (x^2 + y^2 - 1)/(x^2 + y^2 + 1))
on V: (xp, yp) |--> (X, Y, Z) = (2*xp/(xp^2 + yp^2 + 1), 2*yp/(xp^2 + yp^2 + 1), -(xp^2 + yp^2 - 1)/(xp^2 + yp^2 + 1))

We may use $\Phi$ for graphical purposes, for instance to display the grids of the stereographic charts `XU` (in red) and `XV` (in green), with the point $p$ atop:

In [25]:
graph = XU.plot(chart=XR3, mapping=Phi, number_values=25, 
                label_axes=False) + \
        XV.plot(chart=XR3, mapping=Phi, number_values=25, 
                color='green', label_axes=False) + \
        p.plot(chart=XR3, mapping=Phi, label_offset=0.05)
show(graph, viewer='threejs', online=True)

Graphics3d Object

## Scalar fields

Let us define a scalar field, i.e. a smooth map $M\to \mathbb{R}$:

In [26]:
f = M.scalar_field({XU: 1/(1+x^2+y^2), XV: (xp^2+yp^2)/(1+xp^2+yp^2)},
                   name='f')
f.display()

f: M --> R
on U: (x, y) |--> 1/(x^2 + y^2 + 1)
on V: (xp, yp) |--> (xp^2 + yp^2)/(xp^2 + yp^2 + 1)

Scalar fields are implemented by the class `DiffScalarField`
and the function chart representations are
stored in the attribute `_express` of this class, which is a
Python dictionary whose keys are the various charts defined on $M$:

In [27]:
f._express

{Chart (V, (xp, yp)): (xp^2 + yp^2)/(xp^2 + yp^2 + 1),
 Chart (U, (x, y)): 1/(x^2 + y^2 + 1)}

One may wonder about the compatibility of the two coordinate expressions
provided in the definition of $f$. Actually, to enforce the compatibility, it is possible to declare the scalar field in a single chart, `XU` say,
and then to obtain its expression in chart `XV` by analytic continuation
from the expression in $W=U\cap V$, where both expressions are known, thanks
to the transition map `XV` $\to$ `XU`:

In [28]:
f0 = M.scalar_field({XU: 1/(1+x^2+y^2)})
f0.add_expr_by_continuation(XV, U.intersection(V))
f == f0

True

The representation of the scalar field in a given chart, i.e. the public access to the directory `_express`, is obtained via the method `coord_function()`:

In [29]:
fU = f.coord_function(XU)
fU.display()

(x, y) |--> 1/(x^2 + y^2 + 1)

In [30]:
fV = f.coord_function(XV)
fV.display()

(xp, yp) |--> (xp^2 + yp^2)/(xp^2 + yp^2 + 1)

Both `fU` and `fV` are instances of the class `ChartFunction`:

In [31]:
isinstance(fU, sage.manifolds.chart_func.ChartFunction)

True

Mathematically, **chart functions** are real-valued functions on the codomain of the considered chart. They map coordinates to elements of the base field (here $\mathbb{R}$):

In [32]:
fU(1,2)

1/6

In [33]:
fU(*XU(p))

1/6

while scalar fields maps manifold points to $\mathbb{R}$:

In [34]:
f(p)

1/6

Internally, each chart function stores coordinate expressions with respect to various computational engines:

- SageMath symbolic engine, based on the [Pynac](http://pynac.org) backend, 
  with   Maxima used for some simplifications or computation of integrals;
- [SymPy](http://www.sympy.org}) (Python library for symbolic mathematics);
- in the future, more symbolic engines (e.g. Giac) or numerical ones will be implemented

The coordinate expressions are stored in the dictionary `_express`,
whose keys are strings identifying the computational engines. By default
only SageMath symbolic expressions, i.e. expressions pertaining
to the so-called Symbolic Ring (`SR`), are stored:

In [35]:
fU._express

{'SR': 1/(x^2 + y^2 + 1)}

The public access to the dictionary `_express` is performed via the
method `expr()`:

In [36]:
fU.expr()

1/(x^2 + y^2 + 1)

In [37]:
type(fU.expr())

<type 'sage.symbolic.expression.Expression'>

Actually, `fU.expr()` is a shortcut for `fU.expr('SR')` since
`SR` is the default symbolic engine. Note that the class
`Expression` is that devoted to SageMath symbolic expressions.
The method `expr()` can also be invoked to get the expression in
another symbolic engine, for instance SymPy:

In [38]:
fU.expr('sympy')

1/(x**2 + y**2 + 1)

In [39]:
type(fU.expr('sympy'))

<class 'sympy.core.power.Pow'>

This operation has updated the dictionary `_express`:

In [40]:
fU._express

{'SR': 1/(x^2 + y^2 + 1), 'sympy': 1/(x**2 + y**2 + 1)}

The default calculus engine for chart functions of chart `XU` can
changed thanks to the method `set_calculus_method()`:

In [41]:
XU.set_calculus_method('sympy')
fU.expr()

1/(x**2 + y**2 + 1)

We can have better (i.e. LaTeX) rendering of SymPy objects with
```python
from sympy import init_printing
init_printing()
```
We don't use it here to make clear the distinction between `SR` objects and SymPy ones.

Reverting to SageMath's symbolic engine:

In [42]:
XU.set_calculus_method('SR')
fU.expr()

1/(x^2 + y^2 + 1)

Symbolic expressions can be accessed directly from the scalar field,
`f.expr(XU)` being a shortcut for `f.coord_function(XU).expr()`:

In [43]:
f.expr(XU)

1/(x^2 + y^2 + 1)

In [44]:
f.expr(XV)

(xp^2 + yp^2)/(xp^2 + yp^2 + 1)

## Algebra of scalar fields

The commutative algebra of scalar fields on $M$ is

In [45]:
CM = M.scalar_field_algebra()
CM

Algebra of differentiable scalar fields on the 2-dimensional differentiable manifold M

In [46]:
CM.category()

Category of commutative algebras over Symbolic Ring

As for the manifold classes, the actual Python class implementing
$C^\infty(M)$ is inherited from `DiffScalarFieldAlgebra` via the
[category mechanism](http://doc.sagemath.org/html/en/reference/categories/sage/categories/primer.html), hence it bares the name `DiffScalarFieldAlgebra-with-category`:

In [47]:
type(CM)

<class 'sage.manifolds.differentiable.scalarfield_algebra.DiffScalarFieldAlgebra_with_category'>

The class `DiffScalarFieldAlgebra-with-category` is dynamically generated as a subclass of `DiffScalarFieldAlgebra` with extra functionalities, like
for instance the method `is_commutative()`:

In [48]:
CM.is_commutative()

True

Let us have a look at the code of this method; it suffices to use the double question mark:

In [49]:
CM.is_commutative??

We see from the `File` field that the code belongs to the category part of SageMath, not to the manifold part, where the class `DiffScalarFieldAlgebra` is defined.

Regarding the scalar field $f$ introduced above, we have of course

In [50]:
f in CM

True

Actually, in Sage language, `CM`=$C^\infty(M)$ is the parent of `f`:

In [51]:
f.parent() is CM

True

The zero element of the algebra $C^\infty(M)$ is

In [52]:
CM.zero().display()

zero: M --> R
on U: (x, y) |--> 0
on V: (xp, yp) |--> 0

while its unit element is

In [53]:
CM.one().display()

1: M --> R
on U: (x, y) |--> 1
on V: (xp, yp) |--> 1

## Implementation of algebraic operations on scalar fields

Let us consider some operation in the algebra $C^\infty(M)$:

In [54]:
h = f + 2*CM.one()
h.display()

M --> R
on U: (x, y) |--> (2*x^2 + 2*y^2 + 3)/(x^2 + y^2 + 1)
on V: (xp, yp) |--> (3*xp^2 + 3*yp^2 + 2)/(xp^2 + yp^2 + 1)

In [55]:
h(p)

13/6

We are going to investigate how the above addition is performed. For the Python interpreter
`h = f + 2*CM.one()` is equivalent to `h = f.__add__(2*CM.one())`,
i.e. the `+` operator amounts to calling the method `__add__()` on `f`.
To have a look at the source code of this method, we use the double question mark:

In [56]:
f.__add__??

We see that the method `__add__()` is implemented at the level
of the class `Element` from which `DiffScalarField` inherits, via
`CommutativeAlgebraElement`.
In the present case, `left` = `f` and `right` = `2*CM.one()`
have the same parent, namely the algebra `CM`, so that the actual
result is computed in the line
```python
return (<Element>left)._add_(right)
```
This invokes the method `_add_()`
(note the single underscore on each side of `add`). This operator is
implemented at the level of `ScalarField`, as it can be checked from the source code:

In [57]:
f._add_??

This reflects a [general strategy](http://doc.sagemath.org/html/en/thematic_tutorials/coercion_and_categories.html) in SageMath: the arithmetic Python operators `__add__`, `__sub__`,  etc. are implemented at the
top-level class `Element`, while the specific element subclasses,
like `ScalarField` here, implement single-underscore methods `_add_`, `_sub_`, etc., which perform the actual computation
when both operands have the same parent.

Looking at the code, we notice that the first step is to search
for the charts in which both operands of the addition operator have a coordinate
expression. This is performed by the method `common_charts()`;
in the current case, we get the two stereographic charts defined on $M$:

In [58]:
f.common_charts(2*CM.one())

[Chart (U, (x, y)), Chart (V, (xp, yp))]

In general, `common_charts()` returns the charts for which both operands
have already a known coordinate expression or for which a coordinate
expression can be computed by a known transition map, as we can see on the source code:

In [59]:
f.common_charts??

Once the list of charts in which both operands have a coordinate expression,
the addition is performed at the chart function level.
The code for the addition of chart functions defined on the same chart
is (recall that `fU` is the chart function representing $f$ in chart `XU`):

In [60]:
fU._add_??

We notice that the addition is performed on the symbolic expression
with respect to the symbolic engine currently at work (SageMath/Pynac, SymPy,...), as returned by
the method `expr()`.
Let us recall that the user can change the symbolic engine at any time
by means of the method `set_calculus_method()`, applied either to
a chart or to an open subset (possibly `M` itself).
Besides, we notice in the code above that the result of the symbolic addition
is automatically simplified, by means of the method `_simplify`.
It invokes a chain of simplification functions, which depends on the
symbolic engine (See [here](https://github.com/sagemath/sage/blob/develop/src/sage/manifolds/utilities.py)
for details).

Let us now discuss the second case in the `__add__` method of
`Element`, namely the case for which the parents of both operands are
different. This case is treated via SageMath **coercion model**, which allows one to deal with additions like

In [61]:
h1 = f + 2
h1.display()

M --> R
on U: (x, y) |--> (2*x^2 + 2*y^2 + 3)/(x^2 + y^2 + 1)
on V: (xp, yp) |--> (3*xp^2 + 3*yp^2 + 2)/(xp^2 + yp^2 + 1)

A priori, `f + 2` is not a well defined operation, since the integer $2$ does not
belong to the algebra $C^\infty(M)$. However SageMath manages to treat it
because $2$ can be coerced (i.e. automatically and unambigously converted) via `CM(2)`
into a element of $C^\infty(M)$, namely the constant scalar field whose value is $2$:

In [62]:
CM(2).display()

M --> R
on U: (x, y) |--> 2
on V: (xp, yp) |--> 2

This happens because there exists a coercion map from the parent of $2$, namely the ring of integers $\mathbb{Z}$
(denoted `ZZ` in SageMath), to $C^\infty(M)$:

In [63]:
2.parent()

Integer Ring

In [64]:
CM.has_coerce_map_from(ZZ)

True